In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

import kraft

In [ ]:
directory_path = "/home/kwat/garden/data/cancer_cell_line/mutation/"

kraft.path.path(directory_path)

In [ ]:
maf = pd.read_csv("{}/CCLE_mutations.csv".format(directory_path), sep="\t")

maf

In [ ]:
maf = maf.loc[
    :,
    [
        "Hugo_Symbol",
        "Variant_Classification",
        "Genome_Change",
        "Protein_Change",
        "Variant_annotation",
        "DepMap_ID",
    ],
]

maf = maf.loc[maf.loc[:, "Variant_annotation"] == "damaging", :]

maf.loc[:, "Cell Line"] = kraft.cell_line.rename(maf.loc[:, "DepMap_ID"].to_numpy())

maf

In [ ]:
genes = np.unique(maf.loc[:, "Hugo_Symbol"].to_numpy())

gene_to_i = kraft.array.map_int(genes)[0]

print(genes.size)

genes

In [ ]:
def format_(gene_str):

    gene, str_ = gene_str

    if not isinstance(str_, str):

        str_ = ""

    return np.asarray("{}_{}".format(gene, str_), dtype="object")

In [ ]:
variant_classifications = np.unique(
    np.apply_along_axis(
        format_, 1, maf.loc[:, ["Hugo_Symbol", "Variant_Classification"]].to_numpy(),
    )
)

variant_classification_to_i = kraft.array.map_int(variant_classifications)[0]

print(variant_classifications.size)

variant_classifications

In [ ]:
protein_changes = np.unique(
    np.apply_along_axis(
        format_, 1, maf.loc[:, ["Hugo_Symbol", "Protein_Change"]].to_numpy()
    )
)

protein_change_to_i = kraft.array.map_int(protein_changes)[0]

print(protein_changes.size)

protein_changes

In [ ]:
protein_changes = np.unique(
    np.apply_along_axis(
        format_, 1, maf.loc[:, ["Hugo_Symbol", "Protein_Change"]].to_numpy()
    )
)

protein_change_to_i = kraft.array.map_int(protein_changes)[0]

print(protein_changes.size)

protein_changes

In [ ]:
cell_lines = np.unique(maf.loc[:, "Cell Line"].to_numpy())

cell_line_to_i = kraft.array.map_int(cell_lines)[0]

print(cell_lines.size)

cell_lines

In [ ]:
gene_x_cell_line = np.full((genes.size, cell_lines.size), 0)

variant_classification_x_cell_line = np.full(
    (variant_classifications.size, cell_lines.size), 0
)

protein_change_x_cell_line = np.full((protein_changes.size, cell_lines.size), 0)

In [ ]:
for gene, variant_classification, protein_change, cell_line in maf.loc[
    :, ["Hugo_Symbol", "Variant_Classification", "Protein_Change", "Cell Line"]
].to_numpy():

    gene_i = gene_to_i[gene]

    cell_line_i = cell_line_to_i[cell_line]

    gene_x_cell_line[gene_i, cell_line_i] += 1

    if not isinstance(variant_classification, str):

        variant_classification = ""

    variant_classification_i = variant_classification_to_i[
        str(format_((gene, variant_classification)))
    ]

    variant_classification_x_cell_line[variant_classification_i, cell_line_i] += 1

    if not isinstance(protein_change, str):

        protein_change = ""

    protein_change_i = protein_change_to_i[str(format_((gene, protein_change)))]

    protein_change_x_cell_line[protein_change_i, cell_line_i] += 1

In [ ]:
gene_x_cell_line = pd.DataFrame(gene_x_cell_line, index=genes, columns=cell_lines)

gene_x_cell_line.to_csv(
    "{}/gene_x_cell_line_x_n_mutation.tsv".format(directory_path), sep="\t"
)

gene_x_cell_line

In [ ]:
gene_x_cell_line_x_01 = gene_x_cell_line.clip(upper=1)

gene_x_cell_line_x_01.to_csv(
    "{}/gene_x_cell_line_x_mutation_01.tsv".format(directory_path), sep="\t"
)

gene_x_cell_line_x_01

In [ ]:
variant_classification_x_cell_line = pd.DataFrame(
    variant_classification_x_cell_line,
    index=variant_classifications,
    columns=cell_lines,
)

variant_classification_x_cell_line.to_csv(
    "{}/gene_variant_classification_x_cell_line.tsv".format(directory_path), sep="\t"
)

variant_classification_x_cell_line

In [ ]:
protein_change_x_cell_line = pd.DataFrame(
    protein_change_x_cell_line, index=protein_changes, columns=cell_lines
)

protein_change_x_cell_line.to_csv(
    "{}/gene_protein_change_x_cell_line.tsv".format(directory_path), sep="\t"
)

protein_change_x_cell_line